Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [18]:
from itertools import product
from random import random, randint, shuffle, seed, choice
from functools import reduce
from copy import copy
import numpy as np
from scipy import sparse

In [19]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [20]:
x = make_set_covering_problem(1000, 1000, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: False


In [21]:
PROBLEM_SIZE = 1000
NUM_SETS = 1000
DENSITY = 0.3
x = make_set_covering_problem(PROBLEM_SIZE, NUM_SETS, DENSITY)

In [22]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [x.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )
    return valid, -cost

In [23]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

In [31]:
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
print(fitness(current_state))

for step in range(10000):
    new_state = tweak(current_state)
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state
        print(fitness(current_state))

(0, 0)
(296, -1)
(514, -2)
(668, -3)
(767, -4)
(827, -5)
(883, -6)
(920, -7)
(945, -8)
(960, -9)
(979, -10)
(985, -11)
(987, -12)
(992, -13)
(995, -14)
(996, -15)
(999, -16)
(1000, -17)
(1000, -16)
(1000, -15)


In [ ]:
current_state